In [47]:
import cv2
import numpy as np
import pytesseract
import imutils

In [48]:
# Reads the image and resizes it
img = cv2.imread('007.jpg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400) )

In [49]:
# Grayscales
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 55, 55)
# cv2.imshow('gray',gray)

In [50]:
# Binarizes the image 
edged = cv2.Canny(gray, 30, 200)
# cv2.imshow('binarized',edged)

In [51]:
# Grabs the counters using a rectangular box and crops that
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

In [52]:
# Find all the enclosures, and iterate a for loop to find a rectangular enclosure
for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1


In [53]:
# Only the number plate is made visible, and rest is masked
if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)
    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)

# The masked area is cropped and individual characters are segmented
    (x, y) = np.where(mask == 255)
    (topx, topy) = (np.min(x), np.min(y))
    (bottomx, bottomy) = (np.max(x), np.max(y))
    Cropped = gray[topx:bottomx+1, topy:bottomy+1]

# Pytesseract uses OCR here to recognise these charaters
    text = pytesseract.image_to_string(Cropped, config='--psm 11')
    print("The Number Plate is:",text)
    img = cv2.resize(img,(500,300))
    Cropped = cv2.resize(Cropped,(400,200))
#     cv2.imshow('car',img)
#     cv2.imshow('Cropped',Cropped)

else:
    print("Please click a clearer photo")

# Closes the program
# cv2.waitKey(0)
# cv2.destroyAllWindows()

The Number Plate is: ~HR26DK83##

